<a href="https://colab.research.google.com/github/kreshuklab/vem-primer-examples/blob/main/mitochondria/1_data_preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 # vEM-Mitochondria: Data Preparation
 
 Lorem ipsum

## Google Colab

IMPORTANT: Run the next cells until `Download MitoEM` only if you execute this notebook on Google Colab. If you run this notebook locally, you need to set up a python environment with the correct dependencies beforehand, check out [these instructions](https://github.com/kreshuklab/vem-primer-examples#setting-up-conda-environments-advanced).

In [ ]:
# TODO install dependencies with pip

In [ ]:
# mount google drive
from google.colab import drive
drive.mount("/content/gdrive")

## Download MitoEM data and create volume

Lorem ipsum

In [32]:
# imports required for downloading and volume creation
import hashlib
import os
import requests
from glob import glob
from shutil import copyfileobj
from zipfile import ZipFile

import dask
import dask.array as da
import imageio
import numpy as np
import zarr
from tqdm import tqdm

In [33]:
# url and checksum for the data
mitoem_url = "https://www.dropbox.com/s/z41qtu4y735j95e/EM30-H-im.zip?dl=1"
checksum = "98fe259f36a7d8d43f99981b7a0ef8cdeba2ce2615ff91595f428ae57207a041"
tmp_path = "./mito-em-tmp"
os.makedirs(data_root, exist_ok=True)

In [35]:
# RUN THIS IF YOU ARE RUNNING LOCALLY

# this is where we save the data:
# the complete volume
full_output_path = "vem-primer-data/MitoEM-H.ome.zarr"
# a crop of the volume that we can use to speed up downstream analysis
cropped_output_path = "vem-primer-data/MitoEM-H-cropped.ome.zarr"

os.makedirs(os.path.split(full_output_path)[0], exist_ok=True)
os.makedirs(os.path.split(cropped_output_path)[0], exist_ok=True)

In [ ]:
# RUN THIS IF YOU ARE USING COLAB 

# this is where we save the data:
# the complete volume
# full_output_path = "/content/gdrive/MyDrive/vem-primer-data"
full_output_path = "vem-primer-data/MitoEM-H.ome.zarr"
# 
cropped_output_path = "/content/gdrive/MyDrive/vem-primer-data/MitoEM-H-cropped.ome.zarr"

os.makedirs(os.path.split(full_output_path)[0], exist_ok=True)
os.makedirs(os.path.split(cropped_output_path)[0], exist_ok=True)

In [36]:
# check if the data is already fully downloaded and initial volume has been created
# if yes, we skip the following cells
have_mitoem_vol = os.path.exists(full_output_path)

In [37]:
# download the data
zip_path = os.path.join(tmp_path, "MitoEM-H.zip")
if not have_mitoem_vol and not os.path.exists(zip_path):
    with requests.get(mitoem_url, stream=True) as r:
        filesize = int(r.headers.get("Content-Length", 0))
        desc = f"Download {mitoem_url} to {zip_path}"
        with tqdm.wrapattr(r.raw, "read", total=filesize, desc=desc) as r_raw, open(zip_path, "wb") as f:
            copyfileobj(r_raw, f)
    with open(zip_path, "rb") as f:
        this_checksum = hashlib.sha256(f.read()).hexdigest()
    if this_checksum == checksum:
        print("Download to", zip_path, "was successful.")
    else:
        print("The file was downloaded to", zip_path, "but the file is likely corrupted!")
        print("Please remove", zip_path, "and try the download again.")

In [38]:
# unzip the data
unzipped_path = os.path.join(tmp_path, "im")
if not have_mitoem_vol and not os.path.exists(unzipped_path):
    with ZipFile(zip_path, "r") as f:
        f.extractall(path=tmp_path)

# the file paths for all the pngs with image data
image_paths = glob(os.path.join(unzipped_path, "*.png"))
image_paths.sort()
# we only write the last 500 image tiles, which correspond to the MitoEM test data-set
image_paths = image_paths[500:]

In [39]:
# copy the data into a ome.zarr file
# TODO actually use ome.zarr instead of normal zarr
# FIXME this is much more inefficient than in-memory, need a better solution
if not have_mitoem_vol:
    # we use dask to copy the data into the ome.zarr array lazily
    # see https://docs.dask.org/en/stable/array-creation.html
    # https://docs.dask.org/en/latest/generated/dask.array.to_zarr.html
    imread = dask.delayed(imageio.imread, pure=True)
    print("Creating ome.zarr volume from", len(image_paths), "slices")
    images = [imread(path) for path in image_paths]
    
    # find the shape and datatype from the first image
    sample = images[0].compute()

    # load individual images to get a dask array for each image and stack to get a volume
    arrays = [da.from_delayed(im, dtype=sample.dtype, shape=sample.shape)
              for im in images]
    volume = da.stack(arrays, axis=0)
    chunks = (16, 128, 128)
    volume.rechunk(chunks)

    # write to zarr
    da.to_zarr(volume, os.path.join(full_output_path, "s0"))

In [28]:
# copy the data into a ome.zarr file (in memory; implement an efficient version with dask instead)
# TODO actually use ome.zarr instead of normal zarr
if not have_mitoem_vol:
    import z5py
    from tqdm import trange
    
    im0 = imageio.imread(image_paths[0])
    shape = (len(image_paths),) + im0.shape
    volume = np.zeros(shape, dtype=im0.dtype)
    volume[0] = im0
    
    for z in trange(1, len(image_paths)):
        volume[z] = imageio.imread(image_paths[z])
    
    print("Write volume to", full_output_path)
    chunks = (16, 128, 128)
    with z5py.File(full_output_path, "a") as f:
        f.create_dataset("s0", data=volume, compression="gzip", n_threads=4, chunks=chunks)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 499/499 [02:07<00:00,  3.93it/s]


Write volume to ./mito-em-data/MitoEM-H.ome.zarr


## Downscale and crop the data

Lorem ipsum

In [29]:
# TODO use native ome.zarr code as well
# make scale pyramid
from numcodecs import GZip
scale_factors = [[1, 2, 2], [1, 2, 2], [2, 2, 2], [2, 2, 2]]
store = zarr.DirectoryStore(full_output_path)
arrays, pyramid = [], []
current_factor = np.array([1, 1, 1])
with zarr.open(store, mode="a") as f:
    data = da.from_zarr(f["s0"])
    for scale, factor in enumerate(scale_factors, 1):
        current_factor *= np.array(factor)
        factor_dict = {k: v for k, v in enumerate(current_factor)}
        pyramid.append(
            da.coarsen(np.mean, data, factor_dict, trim_excess=True).rechunk(chunks)
        )
        arrays.append(
            f.zeros(name=f"s{scale}", shape=pyramid[-1].shape, chunks=chunks, compressor=GZip())
        )
    da.store(pyramid, arrays, lock=None)

In [30]:
# make a central crop
store_cropped = zarr.DirectoryStore(cropped_output_path)
crop_shape = (20, 1024, 1024)
crop = tuple(slice(sh // 2 - csh // 2, sh // 2 + csh // 2) for sh, csh in zip(data.shape, crop_shape))
with zarr.open(store) as f_in, zarr.open(store_cropped, mode="a") as f_out:
    f_out.create_dataset(name="s0", data=f_in["s0"][crop], chunks=chunks, compression=GZip())
    for scale, factor in enumerate(scale_factors, 1):
        crop = tuple(slice(c.start // sf, c.stop // sf) for c, sf in zip(crop, factor))
        f_out.create_dataset(name=f"s{scale}", data=f_in[f"s{scale}"][crop], chunks=chunks, compression=GZip())

In [5]:
# data conversion to other formats (if necessary):
# export crops for the different resolutions to tifs
with zarr.open(store_cropped, "r") as f:
    for scale in range(len(f)):
        data = f[f"s{scale}"][:]
        imageio.volsave(f"mito-em-cropped-s{scale}.tif", data)